In [1]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

## IO markup

### Train

In [2]:
from modules.data import bert_data

In [3]:
train_df_path = "/home/eartemov/ae/work/factRuEval-2016/dev.csv"
valid_df_path = "/home/eartemov/ae/work/factRuEval-2016/test.csv"

In [4]:
data = bert_data.LearnData.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels4.txt",
    clear_cache=True
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [5]:
from modules.models.bert_models import BERTBiLSTMAttnCRF

In [6]:
model = BERTBiLSTMAttnCRF.create(len(data.train_ds.idx2label), crf_dropout=0.3)

In [7]:
from modules.train.train import NerLearner

In [8]:
num_epochs = 100

In [9]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/fre-BERTBiLSTMAttnCRF-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [10]:
model.get_n_trainable_params()

2629497

In [ ]:
learner.fit(epochs=num_epochs)

### Predict

In [12]:
from modules.data.bert_data import get_data_loader_for_predict

In [13]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [23]:
preds = learner.predict(dl)

In [24]:
from sklearn_crfsuite.metrics import flat_classification_report

In [25]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [26]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [27]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=["I_ORG", "I_PER", "I_LOC"], digits=4)

In [28]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.8639    0.7803    0.8200      3865
       I_PER     0.9535    0.9706    0.9620      2112
       I_LOC     0.9066    0.8356    0.8697      1557

   micro avg     0.8998    0.8451    0.8716      7534
   macro avg     0.9080    0.8622    0.8839      7534
weighted avg     0.8979    0.8451    0.8701      7534



In [29]:
from modules.analyze_utils.main_metrics import precision_recall_f1

In [30]:
results = precision_recall_f1(true_labels, pred_labels)

processed 56409 tokens with 7534 phrases; found: 7076 phrases; correct: 6367.

precision:  89.98%; recall:  84.51%; FB1:  87.16

	LOC: precision:  90.66%; recall:  83.56%; F1:  86.97 1435

	ORG: precision:  86.39%; recall:  78.03%; F1:  82.00 3491

	PER: precision:  95.35%; recall:  97.06%; F1:  96.20 2150


